label's dim: (44906, 121)

train dim:
test dim:


In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GraphSAGE-master

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GraphSAGE-master


In [2]:
%cd eval_scripts/ 
%ls

/content/drive/MyDrive/GraphSAGE-master/eval_scripts
BioGrid_edgelist  citation_eval.py  ppi_eval.py  reddit_eval.py


ppi_eval.py 

In [3]:
from __future__ import print_function
import json
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
from argparse import ArgumentParser




G = json_graph.node_link_graph(json.load(open("/content/drive/MyDrive/GraphSAGE-master/dataset/ppi/ppi-G.json")))
labels = json.load(open("/content/drive/MyDrive/GraphSAGE-master/dataset/ppi/ppi-class_map.json"))
labels = {int(i):l for i, l in labels.items()}
train_ids = [n for n in G.nodes if not G.nodes[n]['val'] and not G.nodes[n]['test']]
test_ids = [n for n in G.nodes if G.nodes[n]['test']]
train_labels = np.array([labels[i] for i in train_ids])
if train_labels.ndim == 1:
    train_labels = np.expand_dims(train_labels, 1)
test_labels = np.array([labels[i] for i in test_ids])

#nx.info(G)
#G.nodes


print(len(test_labels))
print(len(train_labels))
print(len(G.nodes))
print(train_labels.shape)
print(type(G))


5524
44906
56944
(44906, 121)
<class 'networkx.classes.graph.Graph'>


In [4]:
print(type(np.array(train_ids)),np.array(train_ids).shape,np.array(train_ids))
# print(type(train_labels.tolist()),len(train_labels.tolist()))

<class 'numpy.ndarray'> (44906,) [    0     1     2 ... 44903 44904 44905]


In [5]:
nx.write_edgelist(G, 'BioGrid_edgelist')

In [6]:
import pandas as pd
edge_list = pd.read_csv("BioGrid_edgelist", sep = ' ', header = None)
edge_list = edge_list.drop(2, axis=1)

edge_list.rename(columns = {0:'source', 1: 'target'}, inplace = True)
edge_list

,source,target
0,0,372
1,0,1101
2,0,766
3,1,640
4,1,1
...,...,...
818711,56927,56927
818712,56928,56928
818713,56930,56930
818714,56935,56935


GEM-mater/Gem/evaluation/evaluate_node_classificatoin:

In [7]:
try: import cPickle as pickle
except: import pickle
from sklearn import model_selection as sk_ms
from sklearn.multiclass import OneVsRestClassifier as oneVr
from sklearn.linear_model import LogisticRegression as lr
# from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import numpy as np


class TopKRanker(oneVr):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        prediction = np.zeros((X.shape[0], self.classes_.shape[0]))
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-int(k):]].tolist()
            for label in labels:
                prediction[i, label] = 1
        return prediction


def evaluateNodeClassification(train_ids,test_ids,train_labels,test_labels):
    X_train = np.array(train_ids)
    X_test = np.array(test_ids)
    Y_train= np.array(train_labels)
    Y_test = np.array(test_labels)
    try:
        top_k_list = list(Y_test.toarray().sum(axis=1))
    except:
        top_k_list = list(Y_test.sum(axis=1))
    classif2 = TopKRanker(lr())
    classif2.fit(X_train.reshape(-1, 1), Y_train)
    prediction = classif2.predict(X_test.reshape(-1, 1), top_k_list)
    micro = f1_score(Y_test, prediction, average='micro')
    macro = f1_score(Y_test, prediction, average='macro')
    return (micro, macro)


def expNC(X, Y, test_ratio_arr,
          rounds, res_pre, m_summ):
    print('\tNode Classification:')
    summ_file = open('%s_%s.ncsumm' % (res_pre, m_summ), 'w')
    summ_file.write('Method\t%s\n' % ('\t'.join(map(str, test_ratio_arr))))
    micro = [None] * rounds
    macro = [None] * rounds

    # Remove data points with no class
    # nonZeroIndices = np.where(np.any(Y!=0, axis=1))[0]
    # Y = Y[nonZeroIndices, :]
    # X = X[nonZeroIndices, :]
    for round_id in range(rounds):
        micro_round = [None] * len(test_ratio_arr)
        macro_round = [None] * len(test_ratio_arr)
        for i, test_ratio in enumerate(test_ratio_arr):
            micro_round[i], macro_round[i] = evaluateNodeClassification(
                X,
                Y,
                test_ratio
            )
        micro[round_id] = micro_round
        macro[round_id] = macro_round

    summ_file.write('Micro-F1 LR\t%s\n' % ('\t'.join(map(str, micro[0]))))
    summ_file.write('Macro-F1 LR\t%s\n' % ('\t'.join(map(str, macro[0]))))
    summ_file.close()
    pickle.dump([test_ratio_arr, micro, macro],
                open('%s_%s.nc' % (res_pre, m_summ), 'wb'))


In [8]:
train_embeddings = train_ids
test_embeddings = test_ids
tr_lab= train_labels
ts_lab= test_labels

#code from Glove project 
!pip install gensim==4.2.0
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from time import time
import pandas as pd
%cd /content/drive/MyDrive/glove/src/
from pathlib import Path
import os
import argparse
import pickle
import torch
import yaml
from gensim.models.keyedvectors import KeyedVectors
from glove import GloVe
import h5py
try: import cPickle as pickle
except: import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.manifold import TSNE
import sys
sys.path.insert(0, './')
from numpy import savetxt,loadtxt




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/glove/src


In [9]:
# #construct Graph from the edge list:
# # create undirected graph from the edgelist
# G=nx.from_pandas_edgelist(edge_list, source='source', target='target', create_using=nx.Graph())
# # check the basic properties of the graph
# nx.info(G)
# G.nodes
# #Create Random walks:
# # function to generate random walk sequences of nodes for a particular node
# def get_random_walk(node, walk_length):
#     # initialization
#     random_walk_length = [node]
    
#     #loop over to get the nodes visited in a random walk
#     for i in range(walk_length-1):
#         # list of neighbors
#         neighbors = list(G.neighbors(node))
#         # if the same neighbors are present in ranom_walk_length list, then donot add them as new neighbors
#         neighbors = list(set(neighbors) - set(random_walk_length))    
#         if len(neighbors) == 0:
#             break
#         # pick any one neighbor randomly from the neighbors list
#         random_neighbor = random.choice(neighbors)
#         # append that random_neighbor to the random_walk_length list
#         random_walk_length.append(random_neighbor)
#         node = random_neighbor
        
#     return random_walk_length

# all_nodes = list(G.nodes())
# number_of_random_walks = 1#5
# random_walks = []

# for node in tqdm(all_nodes):
#     # number of random walks
#     for i in range(number_of_random_walks):
#         # append the random walk sequence of a node from a specified length
#         random_walks.append(get_random_walk(node, 100))

# type(random_walks)
# #some nodes have random walks of length 3! most 100 to have 2d array all should be the same lenght
# rw=random_walks
# for i in range(len(rw)):
#    if len(rw[i])< 100:
#       a=len(rw[i])
#       for q in range(100-len(rw[i])):
#           xx=rw[i][len(rw[i])-1]
#           # print(rw[i],'xx: ',xx)
#           rw[i].append(xx)
#           # print('after',rw[i],'xx: ',xx)


# savetxt("/content/drive/MyDrive/glove/biogrid_randomwalks.csv", rw, delimiter=",")



In [10]:
rw = loadtxt("/content/drive/MyDrive/glove/biogrid_randomwalks.csv", delimiter=",")


In [23]:
rwX = rw[:,:]
from sklearn.manifold import Isomap
rwX=np.array(rwX)
print(rwX.shape, rw.shape)

(56944, 100) (56944, 100)


Create the weights of the GloVe(implemented in pytorch) through Isomap:

In [ ]:
from scipy.sparse.linalg import isolve
isolve = Isomap(n_components=100)
Isomapped_randomwalks = isolve.fit_transform(rwX)#### Caveat!! here the list indexes are the id of each vector in KeyedVectors its like a dictionary
savetxt("Isomapped_randomwalks.csv",Isomapped_randomwalks)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_isomap.py:324: UserWarning: The number of connected components of the neighbors graph is 3 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [ ]:
print(Isomapped_randomwalks.shape)

After making random walks go to Glove project and create vocabs(dump random walks in a file like sequensial sentences) run Glove create vectors, then come here and keep on evaluation on the out put vector of the glove.

evaluate by f1 measure:

In [ ]:


def load_config():
    config_filepath =  "/content/drive/MyDrive/glove/config.yaml"

    with open(config_filepath) as f:
    # with config_filepath.open() as f:
        config_dict = yaml.load(f, Loader=yaml.FullLoader)
        print('config_dict',config_dict)
    config = argparse.Namespace()
    for key, value in config_dict.items():
        setattr(config, key, value)
    return config


config = load_config()

# Read raw edgelist of vocabs(All random walks on protein edgelist)
with open(os.path.join(config.cooccurrence_dir, "/content/drive/MyDrive/glove/biogrid_vocab.pkl"), "rb") as f:
    vocab = pickle.load(f)

model = GloVe(
    vocab_size=config.vocab_size,
    embedding_size=config.embedding_size,
    x_max=config.x_max,
    alpha=config.alpha
)
model.load_state_dict(torch.load(config.output_filepath))
keyed_vectors = KeyedVectors(vector_size=config.embedding_size)
keyed_vectors.add_vectors(keys=[vocab.get_token(index) for index in range(config.vocab_size)],weights=(model.firstweight.weight.detach()+ model.weight_tilde.weight.detach()).numpy())


micro, macro= evaluateNodeClassification(train_embeddings,test_embeddings,tr_lab,ts_lab)
print(micro, macro)

In [ ]:
expNC